In [1]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [2]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string in Neo4j.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


# Example usage in a Cypher query
input_name1 = "Michael Caine"
input_name2 = "Maggie Gyllenhaal"

query_name1 = generate_full_text_query(input_name1)
query_name2 = generate_full_text_query(input_name2)

In [2]:
from movies import MoviesKG

movies_kg = MoviesKG()
movies_kg.graph.run(
    """MATCH (u:User)-[:LOVED]->(:Movie)<-[:DIRECTED]-(d:Director) 
    WHERE apoc.text.levenshteinSimilarity(d.name, 'Christopher Nolan') >= 0.9 
    RETURN u.location"""
)

(No data)

In [ ]:
movies_kg.graph.run(
    """cypher MATCH (m:Movie)
        WHERE apoc.text.levenshteinSimilarity(m.title, 'The Dark Knight') >= 0.8
        MATCH (m)<-[:DIRECTOR]-(p:Person)-[:DIRECTOR]->(otherMovies:Movie)
        RETURN otherMovies.title"""
)

In [6]:
movies_kg.graph.run(
    "MATCH (m:Movie {title: 'The Dark Knight'})<-[:DIRECTOR]-(p:Person)-[:DIRECTOR]->(otherMovies:Movie) RETURN otherMovies.title"
)

otherMovies.title
Memento
Dunkirk
The Prestige


In [41]:
res = movies_kg.graph.run(
    """
    MATCH (p:User)
WHERE p.name =~ 'User .*'
DETACH DELETE p
    """
)

In [14]:
movies_kg.graph.run(
    "MATCH (m:Movie) WHERE apoc.text.levenshteinSimilarity(m.title, 'Inside Out 2') >= 0.8 RETURN m.title;"
)

m.title
Inside Out 2
Inside Out


In [16]:
movies_kg.graph.run(
    "MATCH (m:Movie) WHERE apoc.text.levenshteinSimilarity(m.title, 'Kingdom of the Planet of the Apes') >= 0.8 RETURN m.title;"
)

m.title
Kingdom of the Planet of the Apes
Rise of the Planet of the Apes


In [15]:
# movies_kg.clear_database()
movies_kg.filter_data()
# movies_kg.build_graph()
movies_kg.filtered_data[1]

{'movie': {'belongs_to_collection': 'Planet of the Apes (Reboot) Collection',
  'budget': 160000000,
  'id': 653346,
  'overview': "Several generations in the future following Caesar's reign, apes are now the dominant species and live harmoniously while humans have been reduced to living in the shadows. As a new tyrannical ape leader builds his empire, one young ape undertakes a harrowing journey that will cause him to question all that he has known about the past and to make choices that will define a future for apes and humans alike.",
  'release_date': '2024-05-08',
  'revenue': 359772773,
  'runtime': 145,
  'tagline': 'No one can stop the reign.',
  'title': 'Kingdom of the Planet of the Apes'},
 'genres': ['Science Fiction', 'Adventure', 'Action'],
 'production_companies': [{'id': 127928,
   'name': '20th Century Studios',
   'origin_country': 'US'},
  {'id': 133024, 'name': 'Oddball Entertainment', 'origin_country': 'US'},
  {'id': 89254, 'name': 'Jason T. Reed Productions', 'or